In [63]:
'''بِسْمِ اللهِ الرَّحْمٰنِ الرَّحِيْمِ'''

'بِسْمِ اللهِ الرَّحْمٰنِ الرَّحِيْمِ'

In [64]:
'''سُبْحَـٰنَكَ لَا عِلْمَ لَنَآ إِلَّا مَا عَلَّمْتَنَآ ۖ إِنَّكَ أَنتَ الْعَلِيمُ الْحَكِيمُ'''

'سُبْحَـٰنَكَ لَا عِلْمَ لَنَآ إِلَّا مَا عَلَّمْتَنَآ ۖ إِنَّكَ أَنتَ الْعَلِيمُ الْحَكِيمُ'

In [65]:
from IPython.core.display import HTML
HTML(r"""
<style>
    * {
        font-family: Monaco;
    }
</style>
""")

In [66]:
import os,sys,warnings,re,math,gc,time
warnings.filterwarnings("ignore")
os.cpu_count()

12

In [67]:
os.environ["POLARS_MAX_THREADS"] = "40"
import numpy as np
import pandas as pd
import polars as pl
import polars.selectors as cs
from glob import glob
from pathlib import Path
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots
pio.templates["mod"] = go.layout.Template(layout=dict(font=dict(family="Monaco",size=15)))
pio.templates.default = "mod+plotly_dark"
from sklearn.metrics import auc,roc_auc_score
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import miceforest as mf
from miceforest import mean_match_shap,mean_match_default,mean_match_fast_cat
%xmode Minimal

Exception reporting mode: Minimal


In [68]:
pl.Config.set_fmt_str_lengths(100)
pl.Config.set_thousands_separator(",")
pl.Config.set_float_precision(2)
pl.Config.set_fmt_float("full")
pl.Config.set_tbl_rows(40)

polars.config.Config

In [69]:
path_to_train = Path("Downloads/credit_risk/train/")
path_to_test = Path("Downloads/credit_risk/test/")
path_to_features = Path("Downloads/feature_definitions.csv")

In [70]:
feat_df = pl.read_csv(path_to_features)
feat_df.head()

Variable,Description
str,str
"""actualdpd_943P""","""Days Past Due (DPD) of previous contract (actual)."""
"""actualdpdtolerance_344P""","""DPD of client with tolerance."""
"""addres_district_368M""","""District of the person's address."""
"""addres_role_871L""","""Role of person's address."""
"""addres_zip_823M""","""Zip code of the address."""


In [71]:
all_train_files = glob("Downloads/credit_risk/train/*.parquet")
all_test_files = glob("Downloads/credit_risk/test/*.parquet")
train_files_df = pl.DataFrame({"index":range(len(all_train_files)),"path":all_train_files})
test_files_df = pl.DataFrame({"index":range(len(all_test_files)),"path":all_test_files})
train_files_df = (
    train_files_df
    .with_columns(
        (pl.col("path").str.split("/").list.get(-1)).alias("filename")
    )
    .sort(by="filename")
)
test_files_df = (
    test_files_df
    .with_columns(
        (pl.col("path").str.split("/").list.get(-1)).alias("filename")
    )
    .sort(by="filename")
)
display(train_files_df.head())
display(test_files_df.head())

index,path,filename
i64,str,str
31,"""Downloads/credit_risk/train/train_applprev_1_0.parquet""","""train_applprev_1_0.parquet"""
13,"""Downloads/credit_risk/train/train_applprev_1_1.parquet""","""train_applprev_1_1.parquet"""
0,"""Downloads/credit_risk/train/train_applprev_2.parquet""","""train_applprev_2.parquet"""
7,"""Downloads/credit_risk/train/train_base.parquet""","""train_base.parquet"""
11,"""Downloads/credit_risk/train/train_credit_bureau_a_1_0.parquet""","""train_credit_bureau_a_1_0.parquet"""


index,path,filename
i64,str,str
31,"""Downloads/credit_risk/test/test_applprev_1_0.parquet""","""test_applprev_1_0.parquet"""
35,"""Downloads/credit_risk/test/test_applprev_1_1.parquet""","""test_applprev_1_1.parquet"""
9,"""Downloads/credit_risk/test/test_applprev_1_2.parquet""","""test_applprev_1_2.parquet"""
1,"""Downloads/credit_risk/test/test_applprev_2.parquet""","""test_applprev_2.parquet"""
33,"""Downloads/credit_risk/test/test_base.parquet""","""test_base.parquet"""


In [72]:
train_files_df.filter(pl.col("filename").str.contains("base"))

index,path,filename
i64,str,str
7,"""Downloads/credit_risk/train/train_base.parquet""","""train_base.parquet"""


In [73]:
train_base = (
    pl.scan_parquet(all_train_files[7])
    .select(
        pl.col("case_id").cast(pl.UInt32).alias("case_id_base"),
        cs.contains("date").str.to_date().alias("date"),
        pl.col("target").cast(pl.UInt8)
    )
    .with_columns(
        pl.col("date").dt.month().alias("month"),
        pl.col("date").dt.week().alias("week"),
        pl.col("date").dt.weekday().alias("weekday")
    )

    .select(~cs.contains(["target","date"]),cs.contains("target"))
)
display(train_base.fetch().head())

case_id_base,month,week,weekday,target
u32,i8,i8,i8,u8
0,1,1,4,0
1,1,1,4,0
2,1,1,5,0
3,1,1,4,0
4,1,1,5,1


In [74]:
test_files_df.filter(pl.col("filename").str.contains("base"))

index,path,filename
i64,str,str
33,"""Downloads/credit_risk/test/test_base.parquet""","""test_base.parquet"""


In [75]:
test_base = (
    pl.scan_parquet(all_test_files[33])
    .select(
        pl.col("case_id").cast(pl.UInt32).alias("case_id_base"),
        cs.contains("date").str.to_date().alias("date"),
    )
    .with_columns(
        pl.col("date").dt.month().alias("month"),
        pl.col("date").dt.week().alias("week"),
        pl.col("date").dt.weekday().alias("weekday"),
        pl.lit(0).cast(pl.UInt8).alias("target")
    )

    .select(~cs.contains(["date"]))
)
display(test_base.fetch().head())
print(test_base.collect().shape)

case_id_base,month,week,weekday,target
u32,i8,i8,i8,u8
"57,543",10,41,2,0
"57,549",10,41,2,0
"57,551",10,41,2,0
"57,552",10,41,3,0
"57,569",10,41,2,0


(10, 5)


In [76]:
out_case_ids = test_base.select("case_id_base").collect()
total_base = (
    pl.concat([
        train_base,
        test_base
    ],
    )
    .sort(by="case_id_base")
)
total_base.fetch().head()

case_id_base,month,week,weekday,target
u32,i8,i8,i8,u8
0,1,1,4,0
1,1,1,4,0
2,1,1,5,0
3,1,1,4,0
4,1,1,5,1


## Case_id, num_group1 and num_group2 relationships

<font size=6>

Goal of this competition is to predict risk quality of clients (or to be precise applications of clients), so client and his/her application is entity (case_id) for which your model predict probability of default (credit score).

- depth=0 (what we call static attributes) are attributes that are aggregated on case_id level. Example can be age of client or gender, 1 records per 1 case_id.

- depth=1, those are attributes where we have several records per client/application. Example can be previous applications or loans in credit bureau register, each client can have from 0 to n records. Therefore, for one case_id there might be several records, and to index them we use num_group1

- depth=2: For some attributes with depth=1 we have more detailed information, for example for previous applications we have data about instalments like date of payments or days past due of each payments. It means for each previous application, you can have 0…n records about instalments/payments. And as an index is used num_group2

To sum up, 1 client can have several previous applications, and each of those previous applications can have several records about instalments, payments, days past due, etc.

- num_group1 : 0 -> 19
- num_group2 : 0 -> 11

In [77]:
def select_imputable_cols(df:pl.LazyFrame,threshold:int=10,stream=False) -> pl.LazyFrame:
    col_list = (
        df
        .collect(streaming=stream)
        .pipe(lambda df: (df.null_count()/df.shape[0])*100)
        .transpose(include_header=True,column_names=["percentage of nulls"])
        .join(
            feat_df,
            left_on="column",
            right_on="Variable",
            how="left"
        )
        .sort(by="percentage of nulls")
        .select(["column","Description","percentage of nulls"])
        .filter(pl.col("percentage of nulls") < threshold)
        ["column"].to_list()
    )
    _ = df.select(col_list)
    gc.collect()
    return _

In [78]:
def rename_cols(df: pl.LazyFrame) -> pl.LazyFrame:
    return (
        df
        .pipe(
            lambda df: df.rename(
                {_:_ + " " + feat_df.filter(pl.col("Variable") == _).select("Description").item() for _ in df.columns if _ not in ["case_id","num_group1","num_group2"]}
            )
        )
    )

In [80]:
train_past_depth_one = (
    pl.scan_parquet(path_to_train/"train_applprev_1_*.parquet",rechunk=True)
    .select(
        pl.col("case_id").cast(pl.UInt32),
        cs.contains("num_group").cast(pl.UInt8),
        cs.boolean().cast(pl.UInt8),
        cs.float().cast(pl.Float32),
        cs.string().cast(pl.Categorical),
    )
)
train_past_depth_one.fetch().head()

column,column_0
str,u32
"""creationdate_885D""",66
"""firstnonzeroinstldate_307D""","652,482"
"""approvaldate_319D""","3,010,294"
"""dateactivated_425D""","3,141,753"
"""employedfrom_700D""","3,886,478"
"""dtlastpmtallstes_3545839D""","4,043,621"
"""dtlastpmt_581D""","4,750,384"


In [18]:
test_files_df.filter(pl.col("filename").str.contains("applprev"))

index,path,filename
i64,str,str
31,"""Downloads/credit_risk/test/test_applprev_1_0.parquet""","""test_applprev_1_0.parquet"""
35,"""Downloads/credit_risk/test/test_applprev_1_1.parquet""","""test_applprev_1_1.parquet"""
9,"""Downloads/credit_risk/test/test_applprev_1_2.parquet""","""test_applprev_1_2.parquet"""
1,"""Downloads/credit_risk/test/test_applprev_2.parquet""","""test_applprev_2.parquet"""


In [19]:
test_past_depth_one = (
    pl.scan_parquet(path_to_test/"test_applprev_1_*.parquet",rechunk=True)
    .select(train_past_depth_one.columns)
    .cast(train_past_depth_one.schema)
)
test_past_depth_one.fetch().head()

case_id,num_group1,isbidproduct_390L,isdebitcard_527L,actualdpd_943P,annuity_853A,byoccupationinc_3656910L,childnum_21L,credacc_actualbalance_314A,credacc_credlmt_575A,credacc_maxhisbal_375A,credacc_minhisbal_90A,credacc_transactions_402L,credamount_590A,currdebt_94A,downpmt_134A,mainoccupationinc_437A,maxdpdtolerance_577P,outstandingdebt_522A,pmtnum_8L,revolvingaccount_394A,tenor_203L,cancelreason_3545846M,credacc_status_367L,credtype_587L,district_544M,education_1138M,familystate_726L,inittransactioncode_279L,postype_4733339M,profession_152M,rejectreason_755M,rejectreasonclient_4145042M,status_219L
u32,u8,u8,u8,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat
"57,543",0,0,null,0.00,"1,519.80",null,null,null,0.00,null,null,null,"37,998.00",null,0.00,"30,000.00",null,null,31.00,null,31.00,"""P94_109_143""",null,"""COL""","""P197_47_166""","""a55475b1""",null,"""POS""","""P149_40_170""","""a55475b1""","""P99_56_166""","""P94_109_143""","""D"""
"57,549",0,0,null,0.00,"2,048.80",null,null,null,0.00,null,null,null,"20,000.00",null,0.00,"12,000.00",null,null,12.00,null,12.00,"""P94_109_143""",null,"""CAL""","""P7_110_89""","""P97_36_170""","""SINGLE""","""CASH""","""P67_102_161""","""a55475b1""","""P45_84_106""","""P94_109_143""","""D"""
"57,551",0,0,null,0.00,"1,871.80",null,null,null,0.00,null,null,null,"20,000.00",null,0.00,"22,000.00",null,null,12.00,null,12.00,"""P94_109_143""",null,"""CAL""","""a55475b1""","""a55475b1""",null,"""CASH""","""P67_102_161""","""a55475b1""","""P99_56_166""","""P94_109_143""","""D"""
"57,551",1,0,null,0.00,"5,401.00",null,null,null,0.00,null,null,null,"60,000.00",null,0.00,"50,000.00",null,null,18.00,null,18.00,"""P94_109_143""",null,"""CAL""","""P197_47_166""","""P97_36_170""","""MARRIED""","""CASH""","""P46_145_78""","""a55475b1""","""P45_84_106""","""P94_109_143""","""D"""
"57,552",0,0,null,0.00,"4,834.20",null,1.00,null,0.00,null,null,null,"50,000.00",null,0.00,"16,000.00",null,null,18.00,null,18.00,"""P94_109_143""",null,"""CAL""","""P11_36_178""","""P97_36_170""","""MARRIED""","""CASH""","""P46_145_78""","""a55475b1""","""P45_84_106""","""P94_109_143""","""D"""


In [20]:
total_past_depth_one = (
    pl.concat(
        [
            train_past_depth_one,
            test_past_depth_one
        ]
    )
    .sort(by="case_id")
    .pipe(rename_cols)
    .pipe(select_imputable_cols)
    .group_by('case_id')
    .agg(
        cs.integer().min(),
        cs.float().max(),
        cs.categorical().drop_nulls().mode().first()
    )
    .with_columns(
        cs.categorical().fill_null(value=cs.categorical().drop_nulls().mode().first()),
        cs.float().fill_null(strategy="mean")
    )
)
total_past_depth_one.fetch().head()

case_id,num_group1,isbidproduct_390L Flag for determining if the product is a cross-sell in previous applications.,actualdpd_943P Days Past Due (DPD) of previous contract (actual).,mainoccupationinc_437A Client's main income amount in their previous application.,credacc_credlmt_575A Credit card credit limit provided for previous applications.,credamount_590A Loan amount or card limit of previous applications.,downpmt_134A Previous application downpayment amount.,annuity_853A Monthly annuity for previous applications.,pmtnum_8L Number of payments made for the previous application.,tenor_203L Number of instalments in the previous application.,cancelreason_3545846M Application cancellation reason.,district_544M District of the address used in the previous loan application.,education_1138M Applicant's education level from their previous application.,postype_4733339M Type of point of sale.,profession_152M Profession of the client during their previous loan application.,rejectreason_755M Reason for previous application rejection.,rejectreasonclient_4145042M Reason for rejection of the client's previous application.,status_219L Previous application status.,credtype_587L Credit type of previous application.,inittransactioncode_279L Type of the initial transaction made in the previous application of the client.
u32,u8,u8,f32,f32,f32,f32,f32,f32,f32,f32,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat
2,0,0,0.00,"8,200.00",0.00,"16,000.00",0.00,"1,682.40",24.00,24.00,"""a55475b1""","""P136_108_173""","""P97_36_170""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""D""","""CAL""","""CASH"""
3,0,0,0.00,"11,000.00",0.00,"59,999.80",0.00,"6,140.00",12.00,12.00,"""P94_109_143""","""P131_33_167""","""P97_36_170""","""a55475b1""","""a55475b1""","""P94_109_143""","""a55475b1""","""D""","""CAL""","""CASH"""
4,0,0,0.00,"16,000.00",0.00,"40,000.00",0.00,"2,556.60",24.00,24.00,"""P24_27_36""","""P194_82_174""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""T""","""CAL""","""CASH"""
5,0,0,0.00,"62,000.00","5,131.54","60,176.47",658.24,"4,540.34",21.24,21.24,"""P85_114_140""","""P54_133_26""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""T""","""CAL""","""CASH"""
6,0,0,0.00,"37,000.00",0.00,"32,000.00",0.00,"4,189.60",24.00,24.00,"""P94_109_143""","""P82_154_182""","""P97_36_170""","""a55475b1""","""a55475b1""","""P94_109_143""","""a55475b1""","""D""","""CAL""","""CASH"""


In [21]:
train_files_df.filter(pl.col("filename").str.contains("applprev_2"))

index,path,filename
i64,str,str
0,"""Downloads/credit_risk/train/train_applprev_2.parquet""","""train_applprev_2.parquet"""


In [22]:
train_past_depth_two = (
    pl.scan_parquet(all_train_files[0])
    .drop(cs.ends_with("D"))
    .select(
        cs.by_name("case_id").cast(pl.UInt32),
        cs.contains("num_group").cast(pl.UInt8),
        cs.boolean().cast(pl.UInt8),
        cs.float().cast(pl.Float32),
        cs.string().cast(pl.Categorical)
    )
)
train_past_depth_two.fetch().head()

case_id,num_group1,num_group2,cacccardblochreas_147M,conts_type_509L,credacc_cards_status_52L
u32,u8,u8,cat,cat,cat
2,1,1,null,"""EMPLOYMENT_PHONE""",null
2,0,1,null,"""EMPLOYMENT_PHONE""",null
2,0,0,null,"""PRIMARY_MOBILE""",null
2,1,0,null,"""PRIMARY_MOBILE""",null
3,0,1,null,"""PRIMARY_MOBILE""",null


In [23]:
test_files_df.filter(pl.col("filename").str.contains("applprev_2"))

index,path,filename
i64,str,str
1,"""Downloads/credit_risk/test/test_applprev_2.parquet""","""test_applprev_2.parquet"""


In [24]:
test_past_depth_two = (
    pl.scan_parquet(all_test_files[1])
    .select(train_past_depth_two.columns)
    .cast(train_past_depth_two.schema)
)
test_past_depth_two.fetch().head()

case_id,num_group1,num_group2,cacccardblochreas_147M,conts_type_509L,credacc_cards_status_52L
u32,u8,u8,cat,cat,cat
"57,543",0,0,null,"""PRIMARY_MOBILE""",null
"57,549",0,2,null,"""PHONE""",null
"57,549",0,0,null,"""PRIMARY_MOBILE""",null
"57,549",0,1,null,"""HOME_PHONE""",null
"57,551",1,1,null,"""PRIMARY_MOBILE""",null


In [25]:
total_past_depth_two = (
    pl.concat(
        [
            train_past_depth_two,
            test_past_depth_two
        ]
    )
    .sort(by="case_id")
    .pipe(rename_cols)
    .pipe(select_imputable_cols)
    .group_by("case_id")
    .agg(
        cs.integer().min(),
        cs.categorical().drop_nulls().mode().first()
    )
)
total_past_depth_two.fetch().head()

case_id,num_group1,num_group2,cacccardblochreas_147M Card blocking reason.
u32,u8,u8,cat
2,0,0,null
3,0,0,null
4,0,0,null
5,0,0,null
6,0,0,null


## Static Files

In [26]:
train_files_df.filter(cs.contains("file").str.contains("train_static_0_"))

index,path,filename
i64,str,str
21,"""Downloads/credit_risk/train/train_static_0_0.parquet""","""train_static_0_0.parquet"""
16,"""Downloads/credit_risk/train/train_static_0_1.parquet""","""train_static_0_1.parquet"""


In [27]:
train_static_base = (
    pl.scan_parquet("Downloads/credit_risk/train/train_static_0_*.parquet")
    .drop(cs.ends_with("D"))
    .select(
        cs.by_name("case_id").cast(pl.UInt32),
        cs.boolean(),
        cs.float().cast(pl.Float32),
        cs.string().cast(pl.Categorical)
    )
)
train_static_base.fetch().head()

case_id,equalitydataagreement_891L,equalityempfrom_62L,isbidproduct_1095L,isbidproductrequest_292L,isdebitcard_729L,opencred_647L,actualdpdtolerance_344P,amtinstpaidbefduel24m_4187115A,annuity_780A,annuitynextmonth_57A,applicationcnt_361L,applications30d_658L,applicationscnt_1086L,applicationscnt_464L,applicationscnt_629L,applicationscnt_867L,avgdbddpdlast24m_3658932P,avgdbddpdlast3m_4187120P,avgdbdtollast24m_4525197P,avgdpdtolclosure24_3658938P,avginstallast24m_3658937A,avglnamtstart24m_4525187A,avgmaxdpdlast9m_3716943P,avgoutstandbalancel6m_4187114A,avgpmtlast12m_4525200A,clientscnt12m_3712952L,clientscnt3m_3712950L,clientscnt6m_3712949L,clientscnt_100L,clientscnt_1022L,clientscnt_1071L,clientscnt_1130L,clientscnt_136L,clientscnt_157L,clientscnt_257L,clientscnt_304L,…,numpmtchanneldd_318L,numrejects9m_859L,pctinstlsallpaidearl3d_427L,pctinstlsallpaidlat10d_839L,pctinstlsallpaidlate1d_3546856L,pctinstlsallpaidlate4d_3546849L,pctinstlsallpaidlate6d_3546844L,pmtnum_254L,posfpd10lastmonth_333P,posfpd30lastmonth_3976960P,posfstqpd30lastmonth_3976962P,price_1097A,sellerplacecnt_915L,sellerplacescnt_216L,sumoutstandtotal_3546847A,sumoutstandtotalest_4493215A,totaldebt_9A,totalsettled_863A,totinstallast1m_4525188A,bankacctype_710L,cardtype_51L,credtype_322L,disbursementtype_67L,inittransactioncode_186L,lastapprcommoditycat_1041M,lastapprcommoditytypec_5251766M,lastcancelreason_561M,lastrejectcommoditycat_161M,lastrejectcommodtypec_5251769M,lastrejectreason_759M,lastrejectreasonclient_4145040M,lastst_736L,paytype1st_925L,paytype_783L,previouscontdistrict_112M,twobodfilling_608L,typesuite_864L
u32,bool,bool,bool,bool,bool,bool,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,…,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat
0,null,null,false,null,null,null,null,null,"1,917.60",0.00,0.00,0.00,0.00,0.00,0.00,0.00,null,null,null,null,null,null,null,null,null,0.00,0.00,0.00,0.00,0.00,0.00,0.00,null,0.00,0.00,0.00,…,0.00,0.00,null,null,null,null,null,24.00,0.00,0.00,null,null,0.00,0.00,null,null,0.00,0.00,null,null,null,"""CAL""","""GBA""","""CASH""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",null,"""OTHER""","""OTHER""","""a55475b1""","""BO""",null
1,null,null,false,null,null,null,null,null,"3,134.00",0.00,0.00,0.00,0.00,0.00,0.00,0.00,null,null,null,null,null,null,null,null,null,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,3.00,0.00,0.00,…,0.00,0.00,null,null,null,null,null,18.00,0.00,0.00,null,null,0.00,0.00,null,null,0.00,0.00,null,null,null,"""CAL""","""GBA""","""CASH""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",null,"""OTHER""","""OTHER""","""a55475b1""","""BO""",null
2,null,null,false,null,null,false,null,null,"4,937.00",0.00,0.00,0.00,0.00,0.00,0.00,0.00,null,null,null,null,null,null,null,null,null,0.00,0.00,0.00,0.00,0.00,0.00,0.00,null,0.00,0.00,0.00,…,0.00,0.00,null,null,null,null,null,36.00,0.00,0.00,null,null,0.00,0.00,null,null,0.00,0.00,null,null,null,"""CAL""","""GBA""","""CASH""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""D""","""OTHER""","""OTHER""","""a55475b1""","""BO""","""AL"""
3,true,true,false,null,null,false,null,null,"4,643.60",0.00,0.00,1.00,0.00,2.00,0.00,1.00,null,null,null,null,null,null,null,null,null,0.00,0.00,0.00,0.00,0.00,0.00,1.00,null,0.00,0.00,0.00,…,0.00,1.00,null,null,null,null,null,12.00,0.00,0.00,null,null,1.00,1.00,null,null,0.00,0.00,null,null,null,"""CAL""","""GBA""","""CASH""","""a55475b1""","""a55475b1""","""P94_109_143""","""a55475b1""","""a55475b1""","""P94_109_143""","""a55475b1""","""D""","""OTHER""","""OTHER""","""a55475b1""","""BO""","""AL"""
4,null,null,false,null,null,false,null,null,"3,390.20",0.00,0.00,1.00,0.00,0.00,0.00,1.00,null,nu

In [28]:
test_files_df.filter(pl.col("filename").str.contains("static"))

index,path,filename
i64,str,str
32,"""Downloads/credit_risk/test/test_static_0_0.parquet""","""test_static_0_0.parquet"""
23,"""Downloads/credit_risk/test/test_static_0_1.parquet""","""test_static_0_1.parquet"""
13,"""Downloads/credit_risk/test/test_static_0_2.parquet""","""test_static_0_2.parquet"""
10,"""Downloads/credit_risk/test/test_static_cb_0.parquet""","""test_static_cb_0.parquet"""


In [29]:
test_static_base = (
    pl.concat(
        [
            (
                pl.scan_parquet("Downloads/credit_risk/test/test_static_0_0.parquet")
                .select(train_static_base.columns)
                .cast(train_static_base.schema)
            ),
            (
                pl.scan_parquet("Downloads/credit_risk/test/test_static_0_1.parquet")
                .select(train_static_base.columns)
                .cast(train_static_base.schema)
            ),
            (
                pl.scan_parquet("Downloads/credit_risk/test/test_static_0_2.parquet")
                .select(train_static_base.columns)
                .cast(train_static_base.schema)
            )
        ],
        rechunk=True
    )
)
test_static_base.fetch().head()

case_id,equalitydataagreement_891L,equalityempfrom_62L,isbidproduct_1095L,isbidproductrequest_292L,isdebitcard_729L,opencred_647L,actualdpdtolerance_344P,amtinstpaidbefduel24m_4187115A,annuity_780A,annuitynextmonth_57A,applicationcnt_361L,applications30d_658L,applicationscnt_1086L,applicationscnt_464L,applicationscnt_629L,applicationscnt_867L,avgdbddpdlast24m_3658932P,avgdbddpdlast3m_4187120P,avgdbdtollast24m_4525197P,avgdpdtolclosure24_3658938P,avginstallast24m_3658937A,avglnamtstart24m_4525187A,avgmaxdpdlast9m_3716943P,avgoutstandbalancel6m_4187114A,avgpmtlast12m_4525200A,clientscnt12m_3712952L,clientscnt3m_3712950L,clientscnt6m_3712949L,clientscnt_100L,clientscnt_1022L,clientscnt_1071L,clientscnt_1130L,clientscnt_136L,clientscnt_157L,clientscnt_257L,clientscnt_304L,…,numpmtchanneldd_318L,numrejects9m_859L,pctinstlsallpaidearl3d_427L,pctinstlsallpaidlat10d_839L,pctinstlsallpaidlate1d_3546856L,pctinstlsallpaidlate4d_3546849L,pctinstlsallpaidlate6d_3546844L,pmtnum_254L,posfpd10lastmonth_333P,posfpd30lastmonth_3976960P,posfstqpd30lastmonth_3976962P,price_1097A,sellerplacecnt_915L,sellerplacescnt_216L,sumoutstandtotal_3546847A,sumoutstandtotalest_4493215A,totaldebt_9A,totalsettled_863A,totinstallast1m_4525188A,bankacctype_710L,cardtype_51L,credtype_322L,disbursementtype_67L,inittransactioncode_186L,lastapprcommoditycat_1041M,lastapprcommoditytypec_5251766M,lastcancelreason_561M,lastrejectcommoditycat_161M,lastrejectcommodtypec_5251769M,lastrejectreason_759M,lastrejectreasonclient_4145040M,lastst_736L,paytype1st_925L,paytype_783L,previouscontdistrict_112M,twobodfilling_608L,typesuite_864L
u32,bool,bool,bool,bool,bool,bool,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,…,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat
"57,543",null,null,false,null,null,false,null,null,"7,637.20",0.00,0.00,0.00,0.00,0.00,0.00,1.00,null,null,null,null,null,null,null,null,null,0.00,0.00,0.00,0.00,0.00,0.00,0.00,null,0.00,0.00,0.00,…,0.00,0.00,null,null,null,null,null,36.00,0.00,0.00,0.00,0.00,0.00,1.00,null,null,0.00,0.00,null,null,null,"""CAL""","""GBA""","""CASH""","""a55475b1""","""a55475b1""","""P94_109_143""","""P159_130_59""","""P75_90_70""","""P99_56_166""","""P94_109_143""","""D""","""OTHER""","""OTHER""","""a55475b1""","""FO""","""AL"""
"57,549",null,null,false,null,null,false,null,null,902.60,0.00,0.00,0.00,0.00,0.00,0.00,0.00,null,null,null,null,null,null,null,null,null,0.00,0.00,0.00,0.00,0.00,0.00,0.00,null,1.00,0.00,0.00,…,0.00,0.00,null,null,null,null,null,12.00,0.00,0.00,1.00,null,0.00,0.00,null,null,0.00,0.00,null,null,null,"""CAL""","""GBA""","""CASH""","""a55475b1""","""a55475b1""","""P94_109_143""","""a55475b1""","""a55475b1""","""P45_84_106""","""P94_109_143""","""D""","""OTHER""","""OTHER""","""a55475b1""","""BO""","""AL"""
"57,551",null,null,false,null,null,false,null,null,"3,610.20",0.00,0.00,0.00,0.00,0.00,0.00,2.00,null,null,null,null,null,null,null,null,null,0.00,0.00,0.00,0.00,0.00,0.00,0.00,null,1.00,0.00,0.00,…,0.00,0.00,null,null,null,null,null,12.00,0.00,0.00,0.00,null,0.00,1.00,null,null,0.00,0.00,null,null,null,"""CAL""","""GBA""","""CASH""","""a55475b1""","""a55475b1""","""P94_109_143""","""a55475b1""","""a55475b1""","""P99_56_166""","""P94_109_143""","""D""","""OTHER""","""OTHER""","""a55475b1""","""BO""","""AL"""
"57,552",null,null,false,null,null,false,null,null,"6,964.40",0.00,0.00,0.00,0.00,0.00,0.00,1.00,null,null,null,null,null,null,null,null,null,0.00,0.00,0.00,0.00,0.00,0.00,1.00,null,0.00,0.00,0.00,…,0.00,0.00,null,null,null,null,null,18.00,0.00,0.00,0.00,null,0.00,0.00,null,null,0.00,0.00,null,"""CA""",null,"""CAL""","""GBA""","""CASH""","""a55475b1""","""a55475b1""","""P94_109_143""","""a55475b1""","""a55475b1""","""P45_84_106""","""P94_109_143""","""D""","""OTHER""","""OTHER""","""a55475b1""","""FO""",null
"57,569",null,null,false,nu

In [30]:
total_static_base = (
    pl.concat(
        [
            train_static_base,
            test_static_base
        ],
        rechunk=True
    )
    .sort(by="case_id")
    .pipe(rename_cols)
    .pipe(select_imputable_cols)
    .with_columns(
        cs.boolean().fill_null(value=cs.boolean().drop_nulls().mode().first()).cast(pl.UInt8),
        cs.float().fill_null(value=cs.float().drop_nulls().mean()),
        cs.categorical().fill_null(value=cs.categorical().drop_nulls().mode().first())
    )
)
total_static_base.fetch().head()

case_id,isbidproduct_1095L Flag indicating if the product is a cross-sell.,annuity_780A Monthly annuity amount.,applicationcnt_361L Number of applications associated with the same email address as the client.,applications30d_658L Number of applications made by the client in the last 30 days.,applicationscnt_1086L Number of applications associated with the same phone number.,applicationscnt_464L Number of applications made in the last 30 days by other clients with the same employer as the applicant.,applicationscnt_629L Number of applications with the same employer in the last 7 days.,applicationscnt_867L Number of applications associated with the same mobile phone.,clientscnt12m_3712952L Number of clients that have used the same mobile phone as the applicant in the past 12 months.,clientscnt3m_3712950L Number of clients who have the same mobile phone number in the last 3 months.,clientscnt6m_3712949L Total number of clients who have used the same mobile number in the last 6 months.,clientscnt_100L Number of applications with matching employer's phone and client's.,clientscnt_1022L Number of clients sharing the same mobile phone.,clientscnt_1071L Number of applications where the alternative phone number matches that of the client.,clientscnt_1130L Number of applications where client's phone number matches the alternative phone contact.,clientscnt_157L Number of clients whose employer has the same phone number as the client.,clientscnt_257L Number of clients that share an alternative phone number with the applicant.,clientscnt_304L Number of clients with the same phone number.,clientscnt_360L Number of clients that have the same alternative phone number and employer's phone number.,clientscnt_493L Number of clients with matching phone numbers for both the employer and the client.,clientscnt_533L Number of clients with same client's and alternative's phone number,clientscnt_887L Number of clients sharing the same employer's phone number.,clientscnt_946L Number of clients with matching mobile and employer's number.,credamount_770A Loan amount or credit card limit.,disbursedcredamount_1113A Disbursed credit amount after consolidation.,downpmt_116A Amount of downpayment.,homephncnt_628L Number of distinct home phones on client's application.,mobilephncnt_593L Number of persons with the same mobile phone number.,numactivecreds_622L Number of active credits.,numactivecredschannel_414L Number of active credits.,numactiverelcontr_750L Number of active revolving credits.,numcontrs3months_479L Number of contracts in last 3 months.,numnotactivated_1143L Number of non-activated credits.,numpmtchanneldd_318L Number of previous loan contracts for the applicant that had direct debit as payment channel.,numrejects9m_859L Number of credit applications that were rejected in the last 9 months.,sellerplacecnt_915L Number of sellerplaces where the same client's document was used.,sellerplacescnt_216L Number of sellerplaces where the same client's mobile phone was used.,lastapprcommoditycat_1041M Commodity category of the last loan applications made by the applicant.,lastapprcommoditytypec_5251766M Commodity type of the last application.,lastcancelreason_561M Cancellation reason of the last application.,lastrejectcommoditycat_161M Category of commodity in the applicant's last rejected application.,lastrejectcommodtypec_5251769M Commodity type of the last rejected application.,lastrejectreason_759M Reason for rejection on the most recent rejected application.,lastrejectreasonclient_4145040M Reason for the client's last loan rejection.,previouscontdistrict_112M Contact district of the client's previous approved application.,credtype_322L Type of credit.,inittransactioncode_186L Transaction type of the initial credit transaction.,totaldebt_9A Total amount of debt.,annuitynextmonth_57A Next month's amount of annuity.,currdebt_22A Current debt amount of the client.,currdebtcredtyperange_828A Current amount of debt of the applicant.,numinstls_657L Number o

In [31]:
train_static_second_base = (
    pl.scan_parquet("Downloads/credit_risk/train/train_static_cb_0.parquet")
    .drop(cs.ends_with("D"))
    .select(
        cs.by_name("case_id").cast(pl.UInt32),
        cs.boolean(),
        cs.float().cast(pl.Float32),
        cs.string().cast(pl.Categorical)
    )
)
train_static_second_base.fetch().head()

case_id,contractssum_5085716L,days120_123L,days180_256L,days30_165L,days360_512L,days90_310L,firstquarter_103L,for3years_128L,for3years_504L,for3years_584L,formonth_118L,formonth_206L,formonth_535L,forquarter_1017L,forquarter_462L,forquarter_634L,fortoday_1092L,forweek_1077L,forweek_528L,forweek_601L,foryear_618L,foryear_818L,foryear_850L,fourthquarter_440L,numberofqueries_373L,pmtaverage_3A,pmtaverage_4527227A,pmtaverage_4955615A,pmtcount_4527229L,pmtcount_4955617L,pmtcount_693L,pmtscount_423L,pmtssum_45A,riskassesment_940T,secondquarter_766L,thirdquarter_1082L,description_5085714M,education_1103M,education_88M,maritalst_385M,maritalst_893M,requesttype_4525192L,riskassesment_302T
u32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,cat,cat,cat,cat,cat,cat,cat
357,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,6.00,"6,301.40",null,null,null,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",null,null
381,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,6.00,"4,019.60",null,null,null,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",null,null
388,null,6.00,8.00,2.00,10.00,4.00,2.00,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,6.00,10.00,null,null,null,null,null,null,6.00,"14,548.00",null,3.00,5.00,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",null,null
405,null,0.00,0.00,0.00,1.00,0.00,0.00,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,4.00,1.00,null,null,null,null,null,null,6.00,"10,498.24",null,2.00,0.00,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",null,null
409,null,2.00,3.00,0.00,3.00,1.00,4.00,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1.00,3.00,null,null,null,null,null,null,7.00,"6,344.88",null,0.00,4.00,"""a55475b1""","""717ddd49""","""a55475b1""","""a7fcb6e5""","""a55475b1""",null,null


In [32]:
test_static_second_base = (
    pl.scan_parquet("Downloads/credit_risk/test/test_static_cb_0.parquet")
    .select(train_static_second_base.columns)
    .cast(train_static_second_base.schema)
)
test_static_second_base.fetch().head()

case_id,contractssum_5085716L,days120_123L,days180_256L,days30_165L,days360_512L,days90_310L,firstquarter_103L,for3years_128L,for3years_504L,for3years_584L,formonth_118L,formonth_206L,formonth_535L,forquarter_1017L,forquarter_462L,forquarter_634L,fortoday_1092L,forweek_1077L,forweek_528L,forweek_601L,foryear_618L,foryear_818L,foryear_850L,fourthquarter_440L,numberofqueries_373L,pmtaverage_3A,pmtaverage_4527227A,pmtaverage_4955615A,pmtcount_4527229L,pmtcount_4955617L,pmtcount_693L,pmtscount_423L,pmtssum_45A,riskassesment_940T,secondquarter_766L,thirdquarter_1082L,description_5085714M,education_1103M,education_88M,maritalst_385M,maritalst_893M,requesttype_4525192L,riskassesment_302T
u32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,cat,cat,cat,cat,cat,cat,cat
"57,543","22,130.26",9.00,9.00,9.00,10.00,9.00,4.00,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,4.00,10.00,null,null,null,null,null,null,null,null,null,1.00,6.00,"""2fc785b2""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",null,null
"57,549",null,0.00,0.00,0.00,0.00,0.00,0.00,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.00,0.00,null,null,null,null,null,null,null,null,null,2.00,0.00,"""2fc785b2""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",null,null
"57,551",null,2.00,2.00,1.00,2.00,2.00,0.00,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1.00,2.00,null,null,null,null,null,null,null,null,null,0.00,2.00,"""2fc785b2""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",null,null
"57,552",null,0.00,0.00,0.00,0.00,0.00,0.00,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.00,0.00,null,null,"16,327.00",null,14.00,null,null,null,null,0.00,0.00,"""2fc785b2""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",null,null
"57,569",null,0.00,0.00,0.00,0.00,0.00,0.00,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.00,0.00,null,null,"16,303.40",null,14.00,null,null,null,null,1.00,0.00,"""2fc785b2""","""a55475b1""","""a55475b1""","""b6cabe76""","""a55475b1""",null,null


In [33]:
total_static_second_base = (
    pl.concat(
        [
            train_static_second_base,
            test_static_second_base
        ],
        rechunk=True
    )
    .sort(by="case_id")
    .pipe(rename_cols)
    .pipe(select_imputable_cols)
    .with_columns(
        cs.boolean().fill_null(value=cs.boolean().drop_nulls().mode().first()).cast(pl.UInt8),
        cs.float().fill_null(value=cs.float().drop_nulls().mean()),
        cs.categorical().fill_null(value=cs.categorical().drop_nulls().mode().first())
    )
)
total_static_second_base.fetch().head()

case_id,description_5085714M Categorization of clients by credit bureau.,education_1103M Level of education of the client provided by external source.,education_88M Education level of the client.,maritalst_385M Marital status of the client.,maritalst_893M Marital status of the client,days120_123L Number of credit bureau queries for the last 120 days.,days180_256L Number of credit bureau queries for last 180 days.,days30_165L Number of credit bureau queries for the last 30 days.,days360_512L Number of Credit Bureau queries for last 360 days.,days90_310L Number of credit bureau queries for the last 90 days.,firstquarter_103L Number of results obtained from credit bureau in the first quarter.,fourthquarter_440L Number of results in fourth quarter.,numberofqueries_373L Number of queries to credit bureau.,secondquarter_766L Number of results in second quarter.,thirdquarter_1082L Number of results in third quarter.
u32,cat,cat,cat,cat,cat,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
357,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",2.10,2.86,0.71,4.83,1.62,1.78,2.37,4.83,1.72,2.24
381,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",2.10,2.86,0.71,4.83,1.62,1.78,2.37,4.83,1.72,2.24
388,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",6.00,8.00,2.00,10.00,4.00,2.00,6.00,10.00,3.00,5.00
405,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",0.00,0.00,0.00,1.00,0.00,0.00,4.00,1.00,2.00,0.00
409,"""a55475b1""","""717ddd49""","""a55475b1""","""a7fcb6e5""","""a55475b1""",2.00,3.00,0.00,3.00,1.00,4.00,1.00,3.00,0.00,4.00


## Train Person

In [34]:
train_files_df.filter(pl.col("filename").str.contains("person"))

index,path,filename
i64,str,str
5,"""Downloads/credit_risk/train/train_person_1.parquet""","""train_person_1.parquet"""
30,"""Downloads/credit_risk/train/train_person_2.parquet""","""train_person_2.parquet"""


In [35]:
train_person_depth_one = (
    pl.scan_parquet(all_train_files[5])
    .drop(cs.ends_with("D"))
    .select(
        cs.by_name("case_id").cast(pl.UInt32),
        cs.by_name("num_group1").cast(pl.UInt8),
        cs.boolean(),
        cs.float().cast(pl.Float32),
        cs.string().cast(pl.Categorical()),
    )
)
train_person_depth_one.fetch().head()

case_id,num_group1,contaddr_matchlist_1032L,contaddr_smempladdr_334L,isreference_387L,remitter_829L,safeguarantyflag_411L,childnum_185L,mainoccupationinc_384A,personindex_1023L,persontype_1072L,persontype_792L,contaddr_district_15M,contaddr_zipcode_807M,education_927M,empl_employedtotal_800L,empl_industry_691L,empladdr_district_926M,empladdr_zipcode_114M,familystate_447L,gender_992L,housetype_905L,housingtype_772L,incometype_1044T,language1_981M,maritalst_703L,registaddr_district_1083M,registaddr_zipcode_184M,relationshiptoclient_415T,relationshiptoclient_642T,role_1084L,role_993L,sex_738L,type_25L
u32,u8,bool,bool,bool,bool,bool,f32,f32,f32,f32,f32,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat
0,0,false,false,null,null,true,null,"10,800.00",0.00,1.00,1.00,"""P88_18_84""","""P167_100_165""","""P97_36_170""","""MORE_FIVE""","""OTHER""","""P142_57_166""","""P167_100_165""","""MARRIED""",null,null,null,"""SALARIED_GOVT""","""P10_39_147""",null,"""P88_18_84""","""P167_100_165""",null,null,"""CL""",null,"""F""","""PRIMARY_MOBILE"""
0,1,null,null,null,false,null,null,null,1.00,1.00,4.00,"""a55475b1""","""a55475b1""","""a55475b1""",null,null,"""a55475b1""","""a55475b1""",null,null,null,null,null,"""a55475b1""",null,"""a55475b1""","""a55475b1""","""SPOUSE""",null,"""EM""",null,null,"""PHONE"""
0,2,null,null,null,false,null,null,null,2.00,4.00,5.00,"""a55475b1""","""a55475b1""","""a55475b1""",null,null,"""a55475b1""","""a55475b1""",null,null,null,null,null,"""a55475b1""",null,"""a55475b1""","""a55475b1""","""COLLEAGUE""","""SPOUSE""","""PE""",null,null,"""PHONE"""
0,3,null,null,null,null,null,null,null,null,5.00,null,"""a55475b1""","""a55475b1""","""a55475b1""",null,null,"""a55475b1""","""a55475b1""",null,null,null,null,null,"""a55475b1""",null,"""a55475b1""","""a55475b1""",null,"""COLLEAGUE""","""PE""",null,null,"""PHONE"""
1,0,false,false,null,null,true,null,"10,000.00",0.00,1.00,1.00,"""P103_93_94""","""P176_37_166""","""P97_36_170""","""MORE_FIVE""","""OTHER""","""P49_46_174""","""P160_59_140""","""DIVORCED""",null,null,null,"""SALARIED_GOVT""","""P10_39_147""",null,"""P103_93_94""","""P176_37_166""",null,null,"""CL""",null,"""M""","""PRIMARY_MOBILE"""


In [36]:
test_files_df.filter(pl.col("filename").str.contains("person"))

index,path,filename
i64,str,str
25,"""Downloads/credit_risk/test/test_person_1.parquet""","""test_person_1.parquet"""
29,"""Downloads/credit_risk/test/test_person_2.parquet""","""test_person_2.parquet"""


In [37]:
test_person_depth_one = (
    pl.scan_parquet(all_test_files[25])
    .select(train_person_depth_one.columns)
    .cast(train_person_depth_one.schema)
)
test_person_depth_one.fetch().head()

case_id,num_group1,contaddr_matchlist_1032L,contaddr_smempladdr_334L,isreference_387L,remitter_829L,safeguarantyflag_411L,childnum_185L,mainoccupationinc_384A,personindex_1023L,persontype_1072L,persontype_792L,contaddr_district_15M,contaddr_zipcode_807M,education_927M,empl_employedtotal_800L,empl_industry_691L,empladdr_district_926M,empladdr_zipcode_114M,familystate_447L,gender_992L,housetype_905L,housingtype_772L,incometype_1044T,language1_981M,maritalst_703L,registaddr_district_1083M,registaddr_zipcode_184M,relationshiptoclient_415T,relationshiptoclient_642T,role_1084L,role_993L,sex_738L,type_25L
u32,u8,bool,bool,bool,bool,bool,f32,f32,f32,f32,f32,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat
"57,543",0,false,false,null,null,false,null,"36,000.00",0.00,1.00,1.00,"""P107_30_170""","""P144_138_111""","""P97_36_170""","""MORE_ONE""","""OTHER""","""P16_81_96""","""P144_138_111""","""MARRIED""",null,null,null,"""EMPLOYED""","""P209_127_106""",null,"""P107_30_170""","""P144_138_111""",null,null,"""CL""",null,"""M""","""PRIMARY_MOBILE"""
"57,543",1,null,null,null,false,null,null,null,1.00,1.00,5.00,"""a55475b1""","""a55475b1""","""a55475b1""",null,null,"""a55475b1""","""a55475b1""",null,null,null,null,null,"""a55475b1""",null,"""a55475b1""","""a55475b1""","""OTHER""",null,"""EM""",null,null,"""PHONE"""
"57,543",2,null,null,null,null,null,null,null,null,5.00,null,"""a55475b1""","""a55475b1""","""a55475b1""",null,null,"""a55475b1""","""a55475b1""",null,null,null,null,null,"""a55475b1""",null,"""a55475b1""","""a55475b1""",null,"""OTHER""","""PE""",null,null,"""PHONE"""
"57,549",0,false,false,null,null,false,null,"15,000.00",0.00,1.00,1.00,"""P7_110_89""","""P166_45_172""","""P97_36_170""","""MORE_ONE""","""OTHER""","""P7_110_89""","""P166_45_172""","""SINGLE""",null,null,null,"""EMPLOYED""","""P10_39_147""",null,"""P7_110_89""","""P166_45_172""",null,null,"""EM""",null,"""M""","""PRIMARY_MOBILE"""
"57,549",1,null,null,null,false,null,null,null,1.00,1.00,5.00,"""a55475b1""","""a55475b1""","""a55475b1""",null,null,"""a55475b1""","""a55475b1""",null,null,null,null,null,"""a55475b1""",null,"""a55475b1""","""a55475b1""","""SIBLING""",null,"""EM""",null,null,"""ALTERNATIVE_PHONE"""


In [38]:
total_person_depth_one = (
    pl.concat(
        [
            train_person_depth_one,
            test_person_depth_one
        ]
    )
    .sort(by="case_id")
    .pipe(rename_cols)
    .pipe(select_imputable_cols)
    .group_by("case_id")
    .agg(
        cs.integer().min(),
        cs.boolean().drop_nulls().mode().first().cast(pl.UInt8),
        cs.float().max(),
        cs.categorical().drop_nulls().mode().first()
    )
)
total_person_depth_one.fetch().head()

case_id,num_group1,persontype_1072L Person type.,contaddr_district_15M Zip code of a contact person's address.,contaddr_zipcode_807M Zip code of contact address.,education_927M Education level of the person.,empladdr_district_926M District where the employer's address is located.,empladdr_zipcode_114M Zipcode of employer's address.,language1_981M The primary language of the person.,registaddr_district_1083M District of person's registered address.,registaddr_zipcode_184M Registered address's zip code of a person.,role_1084L Type of contact role.,type_25L Contact type of a person.
u32,u8,f32,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat
0,0,5.00,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""PE""","""PHONE"""
1,0,5.00,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""PE""","""PHONE"""
2,0,5.00,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""PE""","""PRIMARY_MOBILE"""
3,0,4.00,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""PE""","""PHONE"""
4,0,5.00,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""PE""","""PHONE"""


In [39]:
train_person_depth_two = (
    pl.scan_parquet(all_train_files[30])
    .drop(cs.ends_with("D"))
    .select(
        cs.by_name("case_id").cast(pl.UInt32),
        cs.contains("num_group").cast(pl.UInt8),
        cs.boolean(),
        cs.float().cast(pl.Float32),
        cs.string().cast(pl.Categorical()),
    )
)
train_person_depth_two.fetch().head()

case_id,num_group1,num_group2,addres_district_368M,addres_role_871L,addres_zip_823M,conts_role_79M,empls_economicalst_849M,empls_employer_name_740M,relatedpersons_role_762T
u32,u8,u8,cat,cat,cat,cat,cat,cat,cat
5,0,0,"""a55475b1""",null,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",null
6,0,0,"""P55_110_32""","""CONTACT""","""P10_68_40""","""P38_92_157""","""P164_110_33""","""a55475b1""",null
6,0,1,"""P55_110_32""","""PERMANENT""","""P10_68_40""","""a55475b1""","""a55475b1""","""a55475b1""",null
6,1,0,"""P204_92_178""","""CONTACT""","""P65_136_169""","""P38_92_157""","""P164_110_33""","""a55475b1""","""OTHER_RELATIVE"""
6,1,1,"""P191_109_75""","""CONTACT""","""P10_68_40""","""P7_147_157""","""a55475b1""","""a55475b1""","""OTHER_RELATIVE"""


In [40]:
test_person_depth_two = (
    pl.scan_parquet(all_test_files[29])
    .select(train_person_depth_two.columns)
    .cast(train_person_depth_two.schema)
)
test_person_depth_two.fetch().head()

case_id,num_group1,num_group2,addres_district_368M,addres_role_871L,addres_zip_823M,conts_role_79M,empls_economicalst_849M,empls_employer_name_740M,relatedpersons_role_762T
u32,u8,u8,cat,cat,cat,cat,cat,cat,cat
"57,552",0,0,"""a55475b1""",null,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",null
"57,569",0,0,"""a55475b1""",null,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",null
"57,630",0,0,"""a55475b1""",null,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",null
"57,631",0,0,"""a55475b1""",null,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",null
"57,632",0,0,"""a55475b1""",null,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",null


In [41]:
total_person_depth_two = (
    pl.concat(
        [
            train_person_depth_two,
            test_person_depth_two
        ]
    )
    .sort(by="case_id")
    .pipe(rename_cols)
    .pipe(select_imputable_cols)
    .group_by("case_id")
    .agg(
        cs.integer().min(),
        cs.boolean().drop_nulls().mode().first().cast(pl.UInt8),
        cs.categorical().drop_nulls().mode().first()
    )
)
total_person_depth_two.fetch().head()

case_id,num_group1,num_group2,addres_district_368M District of the person's address.,addres_zip_823M Zip code of the address.,conts_role_79M Type of contact role of a person.,"empls_economicalst_849M The economical status of the person (num_group1 - person, num_group2 - employment).","empls_employer_name_740M Employer's name (num_group1 - person, num_group2 - employment)."
u32,u8,u8,cat,cat,cat,cat,cat
5,0,0,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1"""
6,0,0,"""P204_92_178""","""P10_68_40""","""P38_92_157""","""a55475b1""","""a55475b1"""
7,0,0,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1"""
8,0,0,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1"""
9,0,0,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1"""


In [42]:
train_files_df.filter(pl.col("filename").str.contains("_a_1_"))


index,path,filename
i64,str,str
11,"""Downloads/credit_risk/train/train_credit_bureau_a_1_0.parquet""","""train_credit_bureau_a_1_0.parquet"""
14,"""Downloads/credit_risk/train/train_credit_bureau_a_1_1.parquet""","""train_credit_bureau_a_1_1.parquet"""
28,"""Downloads/credit_risk/train/train_credit_bureau_a_1_2.parquet""","""train_credit_bureau_a_1_2.parquet"""
9,"""Downloads/credit_risk/train/train_credit_bureau_a_1_3.parquet""","""train_credit_bureau_a_1_3.parquet"""


In [43]:
train_credit_bureau = (
    pl.scan_parquet("Downloads/credit_risk/train/train_credit_bureau_a_1_*.parquet")
    .drop(cs.ends_with("D"))
    .select(
        cs.by_name("case_id").cast(pl.UInt32),
        cs.contains("num_group").cast(pl.UInt16),
        cs.boolean(),
        cs.float().cast(pl.Float32),
        cs.string().cast(pl.Categorical)
    )
)
train_credit_bureau.fetch().head()

case_id,num_group1,annualeffectiverate_199L,annualeffectiverate_63L,contractsum_5085717L,credlmt_230A,credlmt_935A,debtoutstand_525A,debtoverdue_47A,dpdmax_139P,dpdmax_757P,dpdmaxdatemonth_442T,dpdmaxdatemonth_89T,dpdmaxdateyear_596T,dpdmaxdateyear_896T,instlamount_768A,instlamount_852A,interestrate_508L,monthlyinstlamount_332A,monthlyinstlamount_674A,nominalrate_281L,nominalrate_498L,numberofcontrsvalue_258L,numberofcontrsvalue_358L,numberofinstls_229L,numberofinstls_320L,numberofoutstandinstls_520L,numberofoutstandinstls_59L,numberofoverdueinstlmax_1039L,numberofoverdueinstlmax_1151L,numberofoverdueinstls_725L,numberofoverdueinstls_834L,outstandingamount_354A,outstandingamount_362A,overdueamount_31A,overdueamount_659A,overdueamountmax2_14A,overdueamountmax2_398A,overdueamountmax_155A,overdueamountmax_35A,overdueamountmaxdatemonth_284T,overdueamountmaxdatemonth_365T,overdueamountmaxdateyear_2T,overdueamountmaxdateyear_994T,periodicityofpmts_1102L,periodicityofpmts_837L,prolongationcount_1120L,prolongationcount_599L,residualamount_488A,residualamount_856A,totalamount_6A,totalamount_996A,totaldebtoverduevalue_178A,totaldebtoverduevalue_718A,totaloutstanddebtvalue_39A,totaloutstanddebtvalue_668A,classificationofcontr_13M,classificationofcontr_400M,contractst_545M,contractst_964M,description_351M,financialinstitution_382M,financialinstitution_591M,purposeofcred_426M,purposeofcred_874M,subjectrole_182M,subjectrole_93M
u32,u16,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat
388,0,null,null,null,null,null,"374,419.50",0.00,0.00,null,null,7.00,"2,018.00",null,null,null,null,"7,811.45",null,null,null,2.00,1.00,null,60.00,null,54.00,0.00,null,0.00,null,null,"260,093.70",null,0.00,0.00,null,0.00,null,null,7.00,"2,018.00",null,null,30.00,null,null,null,null,null,"268,897.62",0.00,0.00,"374,419.50",0.00,"""4408ff0f""","""a55475b1""","""7241344e""","""a55475b1""","""a55475b1""","""a55475b1""","""55b002a9""","""96a8fdfe""","""a55475b1""","""ab3c25cf""","""ab3c25cf"""
388,1,null,null,null,null,"135,806.00",null,null,0.00,null,null,8.00,"2,018.00",null,"8,742.80",null,null,"8,742.80",null,null,null,null,null,null,null,null,null,0.00,null,0.00,null,null,null,null,0.00,0.00,null,0.00,null,null,8.00,"2,018.00",null,null,null,null,null,null,"114,325.80",null,null,null,null,null,null,"""ea6782cc""","""a55475b1""","""7241344e""","""a55475b1""","""a55475b1""","""a55475b1""","""P204_66_73""","""60c73645""","""a55475b1""","""a55475b1""","""a55475b1"""
388,2,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1"""
388,3,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1"""
388,4,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475

In [44]:
test_files_df.filter(pl.col("filename").str.contains("_a_1_"))

index,path,filename
i64,str,str
16,"""Downloads/credit_risk/test/test_credit_bureau_a_1_0.parquet""","""test_credit_bureau_a_1_0.parquet"""
5,"""Downloads/credit_risk/test/test_credit_bureau_a_1_1.parquet""","""test_credit_bureau_a_1_1.parquet"""
21,"""Downloads/credit_risk/test/test_credit_bureau_a_1_2.parquet""","""test_credit_bureau_a_1_2.parquet"""
27,"""Downloads/credit_risk/test/test_credit_bureau_a_1_3.parquet""","""test_credit_bureau_a_1_3.parquet"""
17,"""Downloads/credit_risk/test/test_credit_bureau_a_1_4.parquet""","""test_credit_bureau_a_1_4.parquet"""


In [45]:
test_credit_bureau = (
    pl.concat(
        [
            pl.scan_parquet(all_test_files[16])
            .select(train_credit_bureau.columns)
            .cast(train_credit_bureau.schema),
            pl.scan_parquet(all_test_files[5])
            .select(train_credit_bureau.columns)
            .cast(train_credit_bureau.schema),
            pl.scan_parquet(all_test_files[21])
            .select(train_credit_bureau.columns)
            .cast(train_credit_bureau.schema),
            pl.scan_parquet(all_test_files[27])
            .select(train_credit_bureau.columns)
            .cast(train_credit_bureau.schema),
            pl.scan_parquet(all_test_files[17])
            .select(train_credit_bureau.columns)
            .cast(train_credit_bureau.schema)
        ],
        rechunk=True
    )
)
test_credit_bureau.fetch().head()

case_id,num_group1,annualeffectiverate_199L,annualeffectiverate_63L,contractsum_5085717L,credlmt_230A,credlmt_935A,debtoutstand_525A,debtoverdue_47A,dpdmax_139P,dpdmax_757P,dpdmaxdatemonth_442T,dpdmaxdatemonth_89T,dpdmaxdateyear_596T,dpdmaxdateyear_896T,instlamount_768A,instlamount_852A,interestrate_508L,monthlyinstlamount_332A,monthlyinstlamount_674A,nominalrate_281L,nominalrate_498L,numberofcontrsvalue_258L,numberofcontrsvalue_358L,numberofinstls_229L,numberofinstls_320L,numberofoutstandinstls_520L,numberofoutstandinstls_59L,numberofoverdueinstlmax_1039L,numberofoverdueinstlmax_1151L,numberofoverdueinstls_725L,numberofoverdueinstls_834L,outstandingamount_354A,outstandingamount_362A,overdueamount_31A,overdueamount_659A,overdueamountmax2_14A,overdueamountmax2_398A,overdueamountmax_155A,overdueamountmax_35A,overdueamountmaxdatemonth_284T,overdueamountmaxdatemonth_365T,overdueamountmaxdateyear_2T,overdueamountmaxdateyear_994T,periodicityofpmts_1102L,periodicityofpmts_837L,prolongationcount_1120L,prolongationcount_599L,residualamount_488A,residualamount_856A,totalamount_6A,totalamount_996A,totaldebtoverduevalue_178A,totaldebtoverduevalue_718A,totaloutstanddebtvalue_39A,totaloutstanddebtvalue_668A,classificationofcontr_13M,classificationofcontr_400M,contractst_545M,contractst_964M,description_351M,financialinstitution_382M,financialinstitution_591M,purposeofcred_426M,purposeofcred_874M,subjectrole_182M,subjectrole_93M
u32,u16,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat
"57,543",0,null,null,"22,130.26",null,"128,397.80",0.00,"4,426.05",802.00,null,null,10.00,"2,020.00",null,0.00,null,null,0.00,null,null,null,1.00,null,null,null,null,null,891.00,null,891.00,null,null,null,null,"4,426.05","107,427.52",null,"107,427.52",null,null,11.00,"2,018.00",null,null,null,null,null,null,0.00,null,null,"4,426.05",null,0.00,null,"""ea6782cc""","""a55475b1""","""b919198c""","""a55475b1""","""a55475b1""","""a55475b1""","""b619fa46""","""60c73645""","""a55475b1""","""ab3c25cf""","""a55475b1"""
"57,543",1,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1"""
"57,543",2,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1"""
"57,543",3,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1"""
"57,543",4,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b

In [46]:
total_credit_bureau = (
    pl.concat(
        [
            train_credit_bureau,
            test_credit_bureau
        ],
        rechunk=True
    )
    .sort(by="case_id")
    .pipe(rename_cols)
    .pipe(select_imputable_cols,stream=True)
    .group_by("case_id")
    .agg(
        cs.integer().min(),
        cs.boolean().drop_nulls().mode().first().cast(pl.UInt8),
        cs.float().drop_nulls().mean(),
        cs.categorical().drop_nulls().mode().first()
    )
)
total_credit_bureau.fetch().head()

case_id,num_group1,classificationofcontr_13M Classificiation of the active contract.,classificationofcontr_400M Classificiation of the closed contract.,contractst_545M Contract status.,contractst_964M Contract status of terminated credit contract.,description_351M Categorization of clients by credit bureau.,financialinstitution_382M Name of financial institution that is linked to a closed contract.,financialinstitution_591M Financial institution name of the active contract.,purposeofcred_426M Purpose of credit for active contract.,purposeofcred_874M Purpose of credit on a closed contract.,subjectrole_182M Subject role in active credit contract.,subjectrole_93M Subject role in closed credit contract.
u32,u16,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat
388,0,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1"""
405,0,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1"""
409,0,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1"""
410,0,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1"""
411,0,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1"""


In [47]:
train_credit_bureau_depth = (
    pl.scan_parquet("Downloads/credit_risk/train/train_credit_bureau_a_2_*.parquet")
    .drop(cs.ends_with("D"))
    .select(
        cs.by_name("case_id").cast(pl.UInt32),
        cs.contains("num_group").cast(pl.UInt16),
        cs.boolean(),
        cs.float().cast(pl.Float32),
        cs.string().cast(pl.Categorical)
    )
)
train_credit_bureau_depth.fetch().head()

case_id,num_group1,num_group2,collater_valueofguarantee_1124L,collater_valueofguarantee_876L,pmts_dpd_1073P,pmts_dpd_303P,pmts_month_158T,pmts_month_706T,pmts_overdue_1140A,pmts_overdue_1152A,pmts_year_1139T,pmts_year_507T,collater_typofvalofguarant_298M,collater_typofvalofguarant_407M,collaterals_typeofguarante_359M,collaterals_typeofguarante_669M,subjectroles_name_541M,subjectroles_name_838M
u32,u16,u16,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,cat,cat,cat,cat,cat,cat
388,0,0,0.00,null,null,null,2.00,null,null,null,"2,018.00",null,"""8fd95e4b""","""a55475b1""","""a55475b1""","""c7a5ad39""","""a55475b1""","""ab3c25cf"""
388,0,1,null,null,null,null,3.00,null,null,null,"2,018.00",null,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1"""
388,0,2,null,null,null,null,4.00,null,null,null,"2,018.00",null,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1"""
388,0,3,null,null,null,null,5.00,null,null,null,"2,018.00",null,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1"""
388,0,4,null,null,null,null,6.00,null,null,null,"2,018.00",null,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1"""


In [48]:
test_credit_bureau_depth = (
    pl.scan_parquet("Downloads/credit_risk/test/test_credit_bureau_a_2_*.parquet")
    .select(train_credit_bureau_depth.columns)
    .cast(train_credit_bureau_depth.schema)
)
test_credit_bureau_depth.fetch().head()

case_id,num_group1,num_group2,collater_valueofguarantee_1124L,collater_valueofguarantee_876L,pmts_dpd_1073P,pmts_dpd_303P,pmts_month_158T,pmts_month_706T,pmts_overdue_1140A,pmts_overdue_1152A,pmts_year_1139T,pmts_year_507T,collater_typofvalofguarant_298M,collater_typofvalofguarant_407M,collaterals_typeofguarante_359M,collaterals_typeofguarante_669M,subjectroles_name_541M,subjectroles_name_838M
u32,u16,u16,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,cat,cat,cat,cat,cat,cat
"57,543",0,0,0.00,null,null,null,2.00,null,null,null,"2,018.00",null,"""9a0c095e""","""a55475b1""","""a55475b1""","""c7a5ad39""","""a55475b1""","""ab3c25cf"""
"57,543",0,1,null,null,null,null,3.00,null,null,null,"2,018.00",null,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1"""
"57,543",0,2,null,null,null,null,4.00,null,null,null,"2,018.00",null,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1"""
"57,543",0,3,null,null,null,null,5.00,null,null,null,"2,018.00",null,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1"""
"57,543",0,4,null,null,null,null,6.00,null,null,null,"2,018.00",null,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1"""


In [49]:
total_credit_bureau_depth = (
    pl.concat(
        [
            train_credit_bureau_depth,
            test_credit_bureau_depth
        ]
    )
    .sort(by="case_id")
    .pipe(rename_cols)
    .pipe(select_imputable_cols,stream=True)
    .group_by("case_id")
    .agg(
        cs.integer().min(),
        cs.float().max(),
        cs.categorical().drop_nulls().mode().first()
    )
)
total_credit_bureau_depth.fetch().head()

case_id,num_group1,num_group2,collater_typofvalofguarant_298M Collateral valuation type (active contract).,collater_typofvalofguarant_407M Collateral valuation type (closed contract).,collaterals_typeofguarante_359M Type of collateral that was used as a guarantee for a closed contract.,collaterals_typeofguarante_669M Collateral type for the active contract.,"subjectroles_name_541M Name of subject role in closed credit contract (num_group1 - terminated contract, num_group2 - subject roles).","subjectroles_name_838M Name of subject role in active credit contract (num_group1 - existing contract, num_group2 - subject roles)."
u32,u16,u16,cat,cat,cat,cat,cat,cat
388,0,0,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1"""
405,0,0,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1"""
409,0,0,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1"""
410,0,0,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1"""
411,0,0,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1"""


In [50]:
try:
    for x in globals().keys():
        if "total" in x:
            print(x)
except RuntimeError:
    for x in globals().keys():
        if "total" in x:
            print(x)

total_base
total_past_depth_one
total_past_depth_two
total_static_base
total_static_second_base
total_person_depth_one
total_person_depth_two
total_credit_bureau
total_credit_bureau_depth


In [51]:
total_df = (
    total_base
    .join(
        total_static_base,
        left_on="case_id_base",
        right_on="case_id",
        how="left"
    )
    .join(
        total_static_second_base,
        left_on="case_id_base",
        right_on="case_id",
        how="left"
    )
    .join(
        total_past_depth_one,
        left_on="case_id_base",
        right_on="case_id",
        how="left"
    )
    .join(
        total_past_depth_two,
        left_on="case_id_base",
        right_on="case_id",
        how="left"
    )
    .join(
        total_credit_bureau,
        left_on="case_id_base",
        right_on="case_id",
        how="left"
    )
    .drop(cs.contains("num_group"))
    .select(
        cs.unsigned_integer(),
        cs.signed_integer(),
        cs.boolean(),
        cs.float(),
        cs.categorical()
    )
    .select(
        cs.contains("case_id"),
        ~cs.contains(["case_id","target"]),
        cs.contains("target")
    )
    .with_columns(
        cs.integer().fill_null(strategy="mean"),
        cs.boolean().fill_null(value=cs.boolean().drop_nulls().mode().first()),
        cs.float().fill_null(strategy="mean"),
        cs.categorical().fill_null(value=cs.categorical().drop_nulls().mode().first())
    )
    .collect(streaming=True)
)
total_df.head()

case_id_base,isbidproduct_1095L Flag indicating if the product is a cross-sell.,isbidproduct_390L Flag for determining if the product is a cross-sell in previous applications.,month,week,weekday,annuity_780A Monthly annuity amount.,applicationcnt_361L Number of applications associated with the same email address as the client.,applications30d_658L Number of applications made by the client in the last 30 days.,applicationscnt_1086L Number of applications associated with the same phone number.,applicationscnt_464L Number of applications made in the last 30 days by other clients with the same employer as the applicant.,applicationscnt_629L Number of applications with the same employer in the last 7 days.,applicationscnt_867L Number of applications associated with the same mobile phone.,clientscnt12m_3712952L Number of clients that have used the same mobile phone as the applicant in the past 12 months.,clientscnt3m_3712950L Number of clients who have the same mobile phone number in the last 3 months.,clientscnt6m_3712949L Total number of clients who have used the same mobile number in the last 6 months.,clientscnt_100L Number of applications with matching employer's phone and client's.,clientscnt_1022L Number of clients sharing the same mobile phone.,clientscnt_1071L Number of applications where the alternative phone number matches that of the client.,clientscnt_1130L Number of applications where client's phone number matches the alternative phone contact.,clientscnt_157L Number of clients whose employer has the same phone number as the client.,clientscnt_257L Number of clients that share an alternative phone number with the applicant.,clientscnt_304L Number of clients with the same phone number.,clientscnt_360L Number of clients that have the same alternative phone number and employer's phone number.,clientscnt_493L Number of clients with matching phone numbers for both the employer and the client.,clientscnt_533L Number of clients with same client's and alternative's phone number,clientscnt_887L Number of clients sharing the same employer's phone number.,clientscnt_946L Number of clients with matching mobile and employer's number.,credamount_770A Loan amount or credit card limit.,disbursedcredamount_1113A Disbursed credit amount after consolidation.,downpmt_116A Amount of downpayment.,homephncnt_628L Number of distinct home phones on client's application.,mobilephncnt_593L Number of persons with the same mobile phone number.,numactivecreds_622L Number of active credits.,numactivecredschannel_414L Number of active credits.,numactiverelcontr_750L Number of active revolving credits.,numcontrs3months_479L Number of contracts in last 3 months.,…,rejectreason_755M Reason for previous application rejection._P94_109_143,rejectreason_755M Reason for previous application rejection._P99_56_166,rejectreason_755M Reason for previous application rejection._a55475b1,rejectreasonclient_4145042M Reason for rejection of the client's previous application._P129_162_80,rejectreasonclient_4145042M Reason for rejection of the client's previous application._P19_25_34,rejectreasonclient_4145042M Reason for rejection of the client's previous application._P30_86_84,rejectreasonclient_4145042M Reason for rejection of the client's previous application._P52_67_90,rejectreasonclient_4145042M Reason for rejection of the client's previous application._P53_10_15,rejectreasonclient_4145042M Reason for rejection of the client's previous application._P59_114_135,rejectreasonclient_4145042M Reason for rejection of the client's previous application._P5_143_178,rejectreasonclient_4145042M Reason for rejection of the client's previous application._P64_121_167,rejectreasonclient_4145042M Reason for rejection of the client's previous application._P69_72_116,rejectreasonclient_4145042M Reason for rejection of the client's previous application._P84_14_61,rejectreasonclient_4145042M Reason for rejection of the client's previous application._P94_109_143,rejectreasonclient_41

In [52]:
submission_df = total_df.filter(pl.col("case_id_base").is_in(out_case_ids))
train_df = total_df.filter(~pl.col("case_id_base").is_in(out_case_ids))
display(submission_df.head(2))
train_df.head()

case_id_base,isbidproduct_1095L Flag indicating if the product is a cross-sell.,isbidproduct_390L Flag for determining if the product is a cross-sell in previous applications.,month,week,weekday,annuity_780A Monthly annuity amount.,applicationcnt_361L Number of applications associated with the same email address as the client.,applications30d_658L Number of applications made by the client in the last 30 days.,applicationscnt_1086L Number of applications associated with the same phone number.,applicationscnt_464L Number of applications made in the last 30 days by other clients with the same employer as the applicant.,applicationscnt_629L Number of applications with the same employer in the last 7 days.,applicationscnt_867L Number of applications associated with the same mobile phone.,clientscnt12m_3712952L Number of clients that have used the same mobile phone as the applicant in the past 12 months.,clientscnt3m_3712950L Number of clients who have the same mobile phone number in the last 3 months.,clientscnt6m_3712949L Total number of clients who have used the same mobile number in the last 6 months.,clientscnt_100L Number of applications with matching employer's phone and client's.,clientscnt_1022L Number of clients sharing the same mobile phone.,clientscnt_1071L Number of applications where the alternative phone number matches that of the client.,clientscnt_1130L Number of applications where client's phone number matches the alternative phone contact.,clientscnt_157L Number of clients whose employer has the same phone number as the client.,clientscnt_257L Number of clients that share an alternative phone number with the applicant.,clientscnt_304L Number of clients with the same phone number.,clientscnt_360L Number of clients that have the same alternative phone number and employer's phone number.,clientscnt_493L Number of clients with matching phone numbers for both the employer and the client.,clientscnt_533L Number of clients with same client's and alternative's phone number,clientscnt_887L Number of clients sharing the same employer's phone number.,clientscnt_946L Number of clients with matching mobile and employer's number.,credamount_770A Loan amount or credit card limit.,disbursedcredamount_1113A Disbursed credit amount after consolidation.,downpmt_116A Amount of downpayment.,homephncnt_628L Number of distinct home phones on client's application.,mobilephncnt_593L Number of persons with the same mobile phone number.,numactivecreds_622L Number of active credits.,numactivecredschannel_414L Number of active credits.,numactiverelcontr_750L Number of active revolving credits.,numcontrs3months_479L Number of contracts in last 3 months.,…,rejectreason_755M Reason for previous application rejection._P94_109_143,rejectreason_755M Reason for previous application rejection._P99_56_166,rejectreason_755M Reason for previous application rejection._a55475b1,rejectreasonclient_4145042M Reason for rejection of the client's previous application._P129_162_80,rejectreasonclient_4145042M Reason for rejection of the client's previous application._P19_25_34,rejectreasonclient_4145042M Reason for rejection of the client's previous application._P30_86_84,rejectreasonclient_4145042M Reason for rejection of the client's previous application._P52_67_90,rejectreasonclient_4145042M Reason for rejection of the client's previous application._P53_10_15,rejectreasonclient_4145042M Reason for rejection of the client's previous application._P59_114_135,rejectreasonclient_4145042M Reason for rejection of the client's previous application._P5_143_178,rejectreasonclient_4145042M Reason for rejection of the client's previous application._P64_121_167,rejectreasonclient_4145042M Reason for rejection of the client's previous application._P69_72_116,rejectreasonclient_4145042M Reason for rejection of the client's previous application._P84_14_61,rejectreasonclient_4145042M Reason for rejection of the client's previous application._P94_109_143,rejectreasonclient_41

case_id_base,isbidproduct_1095L Flag indicating if the product is a cross-sell.,isbidproduct_390L Flag for determining if the product is a cross-sell in previous applications.,month,week,weekday,annuity_780A Monthly annuity amount.,applicationcnt_361L Number of applications associated with the same email address as the client.,applications30d_658L Number of applications made by the client in the last 30 days.,applicationscnt_1086L Number of applications associated with the same phone number.,applicationscnt_464L Number of applications made in the last 30 days by other clients with the same employer as the applicant.,applicationscnt_629L Number of applications with the same employer in the last 7 days.,applicationscnt_867L Number of applications associated with the same mobile phone.,clientscnt12m_3712952L Number of clients that have used the same mobile phone as the applicant in the past 12 months.,clientscnt3m_3712950L Number of clients who have the same mobile phone number in the last 3 months.,clientscnt6m_3712949L Total number of clients who have used the same mobile number in the last 6 months.,clientscnt_100L Number of applications with matching employer's phone and client's.,clientscnt_1022L Number of clients sharing the same mobile phone.,clientscnt_1071L Number of applications where the alternative phone number matches that of the client.,clientscnt_1130L Number of applications where client's phone number matches the alternative phone contact.,clientscnt_157L Number of clients whose employer has the same phone number as the client.,clientscnt_257L Number of clients that share an alternative phone number with the applicant.,clientscnt_304L Number of clients with the same phone number.,clientscnt_360L Number of clients that have the same alternative phone number and employer's phone number.,clientscnt_493L Number of clients with matching phone numbers for both the employer and the client.,clientscnt_533L Number of clients with same client's and alternative's phone number,clientscnt_887L Number of clients sharing the same employer's phone number.,clientscnt_946L Number of clients with matching mobile and employer's number.,credamount_770A Loan amount or credit card limit.,disbursedcredamount_1113A Disbursed credit amount after consolidation.,downpmt_116A Amount of downpayment.,homephncnt_628L Number of distinct home phones on client's application.,mobilephncnt_593L Number of persons with the same mobile phone number.,numactivecreds_622L Number of active credits.,numactivecredschannel_414L Number of active credits.,numactiverelcontr_750L Number of active revolving credits.,numcontrs3months_479L Number of contracts in last 3 months.,…,rejectreason_755M Reason for previous application rejection._P94_109_143,rejectreason_755M Reason for previous application rejection._P99_56_166,rejectreason_755M Reason for previous application rejection._a55475b1,rejectreasonclient_4145042M Reason for rejection of the client's previous application._P129_162_80,rejectreasonclient_4145042M Reason for rejection of the client's previous application._P19_25_34,rejectreasonclient_4145042M Reason for rejection of the client's previous application._P30_86_84,rejectreasonclient_4145042M Reason for rejection of the client's previous application._P52_67_90,rejectreasonclient_4145042M Reason for rejection of the client's previous application._P53_10_15,rejectreasonclient_4145042M Reason for rejection of the client's previous application._P59_114_135,rejectreasonclient_4145042M Reason for rejection of the client's previous application._P5_143_178,rejectreasonclient_4145042M Reason for rejection of the client's previous application._P64_121_167,rejectreasonclient_4145042M Reason for rejection of the client's previous application._P69_72_116,rejectreasonclient_4145042M Reason for rejection of the client's previous application._P84_14_61,rejectreasonclient_4145042M Reason for rejection of the client's previous application._P94_109_143,rejectreasonclient_41

In [53]:
all_case_ids = train_df.select("case_id_base")
train_case_ids,test_case_ids = train_test_split(all_case_ids,test_size=0.25,random_state=52)
valid_case_ids,test_case_ids = train_test_split(test_case_ids,test_size=0.35,random_state=25)

In [54]:
def convert_to_pandas(ids:pl.DataFrame) -> list[pd.DataFrame]:
    return (
        train_df.filter(pl.col("case_id_base").is_in(ids)).select("case_id_base","week","target").to_pandas(),
        train_df.filter(pl.col("case_id_base").is_in(ids)).select(~cs.by_name(["case_id_base","week","target"])).to_pandas(),
        train_df.filter(pl.col("case_id_base").is_in(ids)).select("target").to_pandas()
    )

In [55]:
X_train_base,X_train,y_train = convert_to_pandas(train_case_ids)
X_valid_base,X_valid,y_valid = convert_to_pandas(valid_case_ids)
X_test_base,X_test,y_test = convert_to_pandas(test_case_ids)

In [56]:
display(X_train_base.head(2))
display(X_train.head(2))
display(y_train.head(2))

,case_id_base,week,target
0,0,1,0
1,1,1,0


,isbidproduct_1095L Flag indicating if the product is a cross-sell.,isbidproduct_390L Flag for determining if the product is a cross-sell in previous applications.,month,weekday,annuity_780A Monthly annuity amount.,applicationcnt_361L Number of applications associated with the same email address as the client.,applications30d_658L Number of applications made by the client in the last 30 days.,applicationscnt_1086L Number of applications associated with the same phone number.,applicationscnt_464L Number of applications made in the last 30 days by other clients with the same employer as the applicant.,applicationscnt_629L Number of applications with the same employer in the last 7 days.,...,credtype_587L Credit type of previous application._COL,credtype_587L Credit type of previous application._REL,inittransactioncode_279L Type of the initial transaction made in the previous application of the client._CASH,inittransactioncode_279L Type of the initial transaction made in the previous application of the client._NDF,inittransactioncode_279L Type of the initial transaction made in the previous application of the client._POS,cacccardblochreas_147M Card blocking reason._P17_56_144,cacccardblochreas_147M Card blocking reason._P19_60_110,cacccardblochreas_147M Card blocking reason._P201_63_60,cacccardblochreas_147M Card blocking reason._P33_145_161,cacccardblochreas_147M Card blocking reason._a55475b1
0,0,0,1,4,1917.599976,0.0,0.0,0.0,0.0,0.0,...,1,0,0,0,1,0,0,0,0,1
1,0,0,1,4,3134.000000,0.0,0.0,0.0,0.0,0.0,...,1,0,0,0,1,0,0,0,0,1


,target
0,0
1,0


In [57]:
display(X_valid_base.head(2))
display(X_valid.head(2))
display(y_valid.head(2))

,case_id_base,week,target
0,2,1,0
1,3,1,0


,isbidproduct_1095L Flag indicating if the product is a cross-sell.,isbidproduct_390L Flag for determining if the product is a cross-sell in previous applications.,month,weekday,annuity_780A Monthly annuity amount.,applicationcnt_361L Number of applications associated with the same email address as the client.,applications30d_658L Number of applications made by the client in the last 30 days.,applicationscnt_1086L Number of applications associated with the same phone number.,applicationscnt_464L Number of applications made in the last 30 days by other clients with the same employer as the applicant.,applicationscnt_629L Number of applications with the same employer in the last 7 days.,...,credtype_587L Credit type of previous application._COL,credtype_587L Credit type of previous application._REL,inittransactioncode_279L Type of the initial transaction made in the previous application of the client._CASH,inittransactioncode_279L Type of the initial transaction made in the previous application of the client._NDF,inittransactioncode_279L Type of the initial transaction made in the previous application of the client._POS,cacccardblochreas_147M Card blocking reason._P17_56_144,cacccardblochreas_147M Card blocking reason._P19_60_110,cacccardblochreas_147M Card blocking reason._P201_63_60,cacccardblochreas_147M Card blocking reason._P33_145_161,cacccardblochreas_147M Card blocking reason._a55475b1
0,0,0,1,5,4937.000000,0.0,0.0,0.0,0.0,0.0,...,0,0,1,0,0,0,0,0,0,1
1,0,0,1,4,4643.600098,0.0,1.0,0.0,2.0,0.0,...,0,0,1,0,0,0,0,0,0,1


,target
0,0
1,0


In [58]:
display(X_test_base.head(2))
display(X_test.head(2))
display(y_test.head(2))

,case_id_base,week,target
0,27,1,0
1,29,1,0


,isbidproduct_1095L Flag indicating if the product is a cross-sell.,isbidproduct_390L Flag for determining if the product is a cross-sell in previous applications.,month,weekday,annuity_780A Monthly annuity amount.,applicationcnt_361L Number of applications associated with the same email address as the client.,applications30d_658L Number of applications made by the client in the last 30 days.,applicationscnt_1086L Number of applications associated with the same phone number.,applicationscnt_464L Number of applications made in the last 30 days by other clients with the same employer as the applicant.,applicationscnt_629L Number of applications with the same employer in the last 7 days.,...,credtype_587L Credit type of previous application._COL,credtype_587L Credit type of previous application._REL,inittransactioncode_279L Type of the initial transaction made in the previous application of the client._CASH,inittransactioncode_279L Type of the initial transaction made in the previous application of the client._NDF,inittransactioncode_279L Type of the initial transaction made in the previous application of the client._POS,cacccardblochreas_147M Card blocking reason._P17_56_144,cacccardblochreas_147M Card blocking reason._P19_60_110,cacccardblochreas_147M Card blocking reason._P201_63_60,cacccardblochreas_147M Card blocking reason._P33_145_161,cacccardblochreas_147M Card blocking reason._a55475b1
0,0,0,1,4,1218.000000,0.0,0.0,0.0,0.0,0.0,...,1,0,0,0,1,0,0,0,0,1
1,0,0,1,4,2294.800049,0.0,0.0,8.0,0.0,0.0,...,0,0,1,0,0,0,0,0,0,1


,target
0,0
1,0


In [59]:
lgb_train = lgb.Dataset(X_train,label=y_train)
lgb_valid = lgb.Dataset(X_valid,label=y_valid,reference=lgb_train)

params = {
    "boosting_type":"gbdt",
    "objective": "binary",
    "metric": "auc",
    "max_depth": 16,
    "num_leaves": 64,
    "learning_rate": 0.05,
    "max_bin": 255,
    "colsample_bytree": 0.8,
    "colsample_bynode": 0.8,
    "random_state": 55,
    "reg_alpha": 0.1,
    "reg_lambda": 10,
    "extra_trees": True,
    "bagging_freq": 5,
    "n_estimators": 5000,
    "device": "gpu",
    "verbose": -1
}

gbm = lgb.train(
    params,
    lgb_train,
    valid_sets=lgb_valid,
    callbacks=[lgb.log_evaluation(50)]
)

[50]	valid_0's auc: 0.740976
[100]	valid_0's auc: 0.755117
[150]	valid_0's auc: 0.761577
[200]	valid_0's auc: 0.765369
[250]	valid_0's auc: 0.767602
[300]	valid_0's auc: 0.769319
[350]	valid_0's auc: 0.770418
[400]	valid_0's auc: 0.771336
[450]	valid_0's auc: 0.772393
[500]	valid_0's auc: 0.773171
[550]	valid_0's auc: 0.773633
[600]	valid_0's auc: 0.774251
[650]	valid_0's auc: 0.774678
[700]	valid_0's auc: 0.774946
[750]	valid_0's auc: 0.775147
[800]	valid_0's auc: 0.775369
[850]	valid_0's auc: 0.775573
[900]	valid_0's auc: 0.775771
[950]	valid_0's auc: 0.775939
[1000]	valid_0's auc: 0.776059
[1050]	valid_0's auc: 0.776015
[1100]	valid_0's auc: 0.776137
[1150]	valid_0's auc: 0.776166
[1200]	valid_0's auc: 0.776211
[1250]	valid_0's auc: 0.776252
[1300]	valid_0's auc: 0.776254
[1350]	valid_0's auc: 0.776243
[1400]	valid_0's auc: 0.77631
[1450]	valid_0's auc: 0.776277
[1500]	valid_0's auc: 0.776324
[1550]	valid_0's auc: 0.776313
[1600]	valid_0's auc: 0.77621
[1650]	valid_0's auc: 0.776228

In [60]:
for base, X in [(X_train_base, X_train), (X_valid_base, X_valid), (X_test_base, X_test)]:
    y_pred = gbm.predict(X, num_iteration=gbm.best_iteration)
    base["score"] = y_pred
    
print(f'The AUC score on the train set is: {roc_auc_score(X_train_base["target"], X_train_base["score"])}') 
print(f'The AUC score on the valid set is: {roc_auc_score(X_valid_base["target"], X_valid_base["score"])}') 
print(f'The AUC score on the test set is: {roc_auc_score(X_test_base["target"], X_test_base["score"])}')  

The AUC score on the train set is: 0.9095411978341514
The AUC score on the valid set is: 0.772395003655979
The AUC score on the test set is: 0.770044569684937


In [61]:
def gini_stability(base, w_fallingrate=88.0, w_resstd=-0.5):
    gini_in_time = base.loc[:, ["week", "target", "score"]]\
        .sort_values("week")\
        .groupby("week")[["target", "score"]]\
        .apply(lambda x: 2*roc_auc_score(x["target"], x["score"])-1).tolist()
    
    x = np.arange(len(gini_in_time))
    y = gini_in_time
    a, b = np.polyfit(x, y, 1)
    y_hat = a*x + b
    residuals = y - y_hat
    res_std = np.std(residuals)
    avg_gini = np.mean(gini_in_time)
    return avg_gini + w_fallingrate * min(0, a) + w_resstd * res_std

stability_score_train = gini_stability(X_train_base)
stability_score_valid = gini_stability(X_valid_base)
stability_score_test = gini_stability(X_test_base)

print(f'The stability score on the train set is: {stability_score_train}') 
print(f'The stability score on the valid set is: {stability_score_valid}') 
print(f'The stability score on the test set is: {stability_score_test}')

The stability score on the train set is: 0.8073161458836813
The stability score on the valid set is: 0.5160765205908068
The stability score on the test set is: 0.5019689474098561


In [62]:
out_df = submission_df.select(~cs.by_name(["case_id_base","week","target"])).to_pandas()
y_sub_pred = gbm.predict(out_df)
sub_df = pd.DataFrame({
    "case_id": submission_df.select("case_id_base").to_series().to_numpy(),
    "score": y_sub_pred
}).set_index("case_id")
sub_df.to_csv("submission.csv")